Source: https://www.nasdaq.com/symbol/fb/option-chain?expir=stan&dateindex=-1
Starter code from https://realpython.com/python-web-scraping-practical-introduction/

In [4]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [216]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [217]:
base_url="https://www.nasdaq.com/symbol/luv/option-chain"
url = base_url + "?expir=stan&dateindex=-1"
page_content = simple_get(url)

In [218]:
html = BeautifulSoup(page_content, 'html.parser')

In [219]:
## get stock price
def getStockLatestPrice(html):    
    return float(html.find(id="qwidget_lastsale").string.replace("$", ""))

In [220]:
import re
from datetime import datetime
import pandas as pd
import numpy as np

def getOptionData(html):
    chart_div = html.find("div", class_="OptionsChain-chart")
    chart_headers = []
    for th in chart_div.find_all("th"):
        if th.find("a"):
            chart_headers.append(re.search(r'\".+\"', th.find("a").text)[0].replace('\"', ''))
        else:
            chart_headers.append(th.text)
    ## update for duplicate column names
    seen = []
    for c, col in enumerate(chart_headers):
        if col in seen:
            chart_headers[c] = col + "_2"
        else:
            seen.append(col)
    
    rows = chart_div.findAll("tr")
    rowlist = []
    for i, row in enumerate(rows):
        ## Find rows that have option price data
        ## These rows would have 2 <td>s with links to the option price details for the corresponding call/put, expiry and strike
        results = row.find_all(href=re.compile("option-chain"))
        
        if len(results) > 0:
            #expiry_date = datetime.strptime(results[0].text, '%b %d, %Y')            
            rowlist.append([td.text for td in row.find_all("td")])
    df = pd.DataFrame(rowlist)
    df.columns = chart_headers
    return df

In [221]:
## We're interested in doing covered calls, so just get bid prices for calls (Bid column), Strike, and Calls (expiry date) columns
def getDataForCoveredCalls(df):
    return df[["Calls", "Bid", "Strike"]]


In [224]:
df = getOptionData(html)
print(getStockLatestPrice(html), df)

53.25            Calls Last Sale Change Bid Ask Volume Open Interest Root Strike  \
0   Apr 18, 2019      4.70                     0            15  LUV     48   
1   Apr 18, 2019      4.15                     0             4  LUV   48.5   
2   Apr 18, 2019      4.13                     0            32  LUV     49   
3   Apr 18, 2019      3.50                     0           447  LUV   49.5   
4   Apr 18, 2019      3.58   0.38              3          1085  LUV     50   
5   Apr 18, 2019      2.93   0.38              4            53  LUV   50.5   
6   Apr 18, 2019      2.60   0.39             20           260  LUV     51   
7   Apr 18, 2019      2.15   0.25             13           373  LUV   51.5   
8   Apr 18, 2019      1.75   0.17              7           424  LUV     52   
9   Apr 18, 2019      1.45                    56          2461  LUV   52.5   
10  Apr 18, 2019      1.10   0.04             88           381  LUV     53   
11  Apr 18, 2019      0.88   0.13            261          

In [231]:
new_df["Bid"].replace("", "0")

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
Name: Bid, dtype: object

In [232]:
latestprice = getStockLatestPrice(html)
new_df = getDataForCoveredCalls(df)
new_df.loc[:, "Calls"] = pd.to_datetime(new_df["Calls"])
new_df.loc[:, "MonthsToExpiry"] = (new_df["Calls"] - datetime.today())/np.timedelta64(1, 'M')
new_df.loc[:, "Bid"] = new_df["Bid"].replace("", "0").astype(float)
new_df.loc[:, "Strike"] = new_df["Strike"].astype(float)
new_df.loc[:, "StrikeToCurPricePremium%"] = (new_df["Strike"]*100/latestprice - 100).astype(int)
new_df.loc[:, "Yield%"] = (new_df["Bid"] * 100/latestprice).astype(int)
new_df.loc[:, "AnnualizedYield%"] = (new_df.loc[:, "Yield%"] * 12 / new_df["MonthsToExpiry"]).astype(int)
print(latestprice)
new_df


53.25


,Calls,Bid,Strike,MonthsToExpiry,StrikeToCurPricePremium%,Yield%,AnnualizedYield%
0,2019-04-18,0.0,48.0,0.328667,-9,0,0
1,2019-04-18,0.0,48.5,0.328667,-8,0,0
2,2019-04-18,0.0,49.0,0.328667,-7,0,0
3,2019-04-18,0.0,49.5,0.328667,-7,0,0
4,2019-04-18,0.0,50.0,0.328667,-6,0,0
5,2019-04-18,0.0,50.5,0.328667,-5,0,0
6,2019-04-18,0.0,51.0,0.328667,-4,0,0
7,2019-04-18,0.0,51.5,0.328667,-3,0,0
8,2019-04-18,0.0,52.0,0.328667,-2,0,0
9,2019-04-18,0.0,52.5,0.328667,-1,0,0
